In [29]:
# 🚀 PRINT TOP 5 CHUNKS + FULL RAG ANSWER!
print("🚀 TOP 5 CHUNKS + REAL ANSWER")
print("=" * 70)

# 1. GET TOP 5 CHUNKS (BM25)
print("\n🔍 RETRIEVING TOP 5 CHUNKS...")
from ingestion.retrieval import HybridRetriever

retriever = HybridRetriever(vectordb, docs)
chunks, diagnostics = retriever.retrieve("washer leveling installation", top_k=5, strategy="bm25")

print(f"\n✅ TOP 5 CHUNKS FOUND: {len(chunks)}")
print("\n📄 FULL CHUNKS:")
for i, chunk in enumerate(chunks[:5]):
    print(f"\n--- CHUNK {i+1} ---")
    print(f"SOURCE: {chunk.get('source', 'N/A')}")
    print(f"SCORE: {chunk.get('score', 'N/A')}")
    print(f"TEXT:  {chunk.get('text', '')[:300]}...")
    print()

# 2. CLEAN + GENERATE REAL ANSWER
print("\n🤖 GENERATING FULL RAG ANSWER...")
def clean_text(text):
    import re
    text = re.sub(r'/g\d+/g\d+', '', text)
    text = re.sub(r'[^\w\s\.\,\!\?\-\(\)]', ' ', text)
    return ' '.join(text.split())

clean_chunks = [{"source": c.get('source'), "text": clean_text(c.get('text', '')), "score": c.get('score')} 
                for c in chunks[:5]]

prompt = f"""
Question: How do I level the washer during installation?

TOP 5 CHUNKS FROM YOUR 353 MANUALS:
{json.dumps(clean_chunks, indent=2)}

Answer using ONLY these chunks. Return JSON format.
"""
from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
)

response = model.invoke(prompt)
print("\n✅ FULL RAG ANSWER:")
print(response.content)

print(f"\n🎉 TOP 5 CHUNKS + ANSWER FROM YOUR 353 DOCS!")


🚀 TOP 5 CHUNKS + REAL ANSWER

🔍 RETRIEVING TOP 5 CHUNKS...

✅ TOP 5 CHUNKS FOUND: 5

📄 FULL CHUNKS:

--- CHUNK 1 ---
SOURCE: C:\Users\saich\AppData\Local\Temp\tmpfsefvy3v.pdf
SCORE: 1.0
TEXT:  Set Leveling feet
/g135/g3/g3Loosen the locknuts on 2 front leveling 
feet until you can turn the feet with the 
included wrench. Turn clockwise to 
raise the washer or counterclockwise 
to lower it.
Checking slope
/g135/g3/g3If installation surface is tilted, the 
washer will vibrate. Do not instal...


--- CHUNK 2 ---
SOURCE: C:\Users\saich\AppData\Local\Temp\tmp7nrk89aq.pdf
SCORE: 1.0
TEXT:  Set Leveling feet
/g135/g3/g3Loosen the locknuts on 2 front leveling 
feet until you can turn the feet with the 
included wrench. Turn clockwise to 
raise the washer or counterclockwise 
to lower it.
Checking slope
/g135/g3/g3If installation surface is tilted, the 
washer will vibrate. Do not instal...


--- CHUNK 3 ---
SOURCE: C:\Users\saich\AppData\Local\Temp\tmptn4m4628.pdf
SCORE: 1.0
TEXT:  Set Levelin